In [31]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import optuna
from optuna import Trial
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

from xgboost import XGBRegressor
from xgboost import XGBClassifier
from lightgbm import LGBMRegressor
from lightgbm import LGBMClassifier
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVR
from sklearn.svm import LinearSVC

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
def rmse(x,y):
    return mean_squared_error(x,y)**0.5


from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OrdinalEncoder


def class_score(y_test,preds):
    acc_scor = accuracy_score(y_test, preds)
    log_scor = log_loss(y_test, preds)
    f1_scor = f1_score(y_test, preds)

    return f1_scor

In [32]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [33]:
len(train["person_prefer_d_1"].unique())

1093

In [34]:
train = train.drop(['id',"contents_open_dt","target","person_prefer_f","person_prefer_g"],axis = 1)

In [35]:
test = test.drop(['id',"contents_open_dt","person_prefer_f","person_prefer_g"],axis = 1)

In [36]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501951 entries, 0 to 501950
Data columns (total 30 columns):
 #   Column                  Non-Null Count   Dtype
---  ------                  --------------   -----
 0   d_l_match_yn            501951 non-null  bool 
 1   d_m_match_yn            501951 non-null  bool 
 2   d_s_match_yn            501951 non-null  bool 
 3   h_l_match_yn            501951 non-null  bool 
 4   h_m_match_yn            501951 non-null  bool 
 5   h_s_match_yn            501951 non-null  bool 
 6   person_attribute_a      501951 non-null  int64
 7   person_attribute_a_1    501951 non-null  int64
 8   person_attribute_b      501951 non-null  int64
 9   person_prefer_c         501951 non-null  int64
 10  person_prefer_d_1       501951 non-null  int64
 11  person_prefer_d_2       501951 non-null  int64
 12  person_prefer_d_3       501951 non-null  int64
 13  person_prefer_e         501951 non-null  int64
 14  person_prefer_h_1       501951 non-null  int64
 15  

In [37]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46404 entries, 0 to 46403
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   d_l_match_yn            46404 non-null  bool 
 1   d_m_match_yn            46404 non-null  bool 
 2   d_s_match_yn            46404 non-null  bool 
 3   h_l_match_yn            46404 non-null  bool 
 4   h_m_match_yn            46404 non-null  bool 
 5   h_s_match_yn            46404 non-null  bool 
 6   person_attribute_a      46404 non-null  int64
 7   person_attribute_a_1    46404 non-null  int64
 8   person_attribute_b      46404 non-null  int64
 9   person_prefer_c         46404 non-null  int64
 10  person_prefer_d_1       46404 non-null  int64
 11  person_prefer_d_2       46404 non-null  int64
 12  person_prefer_d_3       46404 non-null  int64
 13  person_prefer_e         46404 non-null  int64
 14  person_prefer_h_1       46404 non-null  int64
 15  person_prefer_h_2  

In [38]:
#pdf보고 전처리
train["person_rn"] = train["person_rn"].astype('object')
train["contents_rn"] = train["contents_rn"].astype('object')

num_cols = train.columns[train.dtypes == 'int64'].to_list()

for i in num_cols:
    train[i] = train[i].astype('category')
cols = ["person_prefer_d_1","person_prefer_d_2","person_prefer_d_3","person_prefer_h_1","person_prefer_h_2","person_prefer_h_3","contents_attribute_l","contents_attribute_d","contents_attribute_h"]
for i in cols:
    train[i] = train[i].astype('object')
    
    
test["person_rn"] = test["person_rn"].astype('object')
test["contents_rn"] = test["contents_rn"].astype('object')

num_cols = test.columns[test.dtypes == 'int64'].to_list()

for i in num_cols:
    test[i] = test[i].astype('category')
cols = ["person_prefer_d_1","person_prefer_d_2","person_prefer_d_3","person_prefer_h_1","person_prefer_h_2","person_prefer_h_3","contents_attribute_l","contents_attribute_d","contents_attribute_h"]
for i in cols:
    test[i] = test[i].astype('object')

In [39]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501951 entries, 0 to 501950
Data columns (total 30 columns):
 #   Column                  Non-Null Count   Dtype   
---  ------                  --------------   -----   
 0   d_l_match_yn            501951 non-null  bool    
 1   d_m_match_yn            501951 non-null  bool    
 2   d_s_match_yn            501951 non-null  bool    
 3   h_l_match_yn            501951 non-null  bool    
 4   h_m_match_yn            501951 non-null  bool    
 5   h_s_match_yn            501951 non-null  bool    
 6   person_attribute_a      501951 non-null  category
 7   person_attribute_a_1    501951 non-null  category
 8   person_attribute_b      501951 non-null  category
 9   person_prefer_c         501951 non-null  category
 10  person_prefer_d_1       501951 non-null  object  
 11  person_prefer_d_2       501951 non-null  object  
 12  person_prefer_d_3       501951 non-null  object  
 13  person_prefer_e         501951 non-null  category
 14  pers

In [40]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46404 entries, 0 to 46403
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   d_l_match_yn            46404 non-null  bool    
 1   d_m_match_yn            46404 non-null  bool    
 2   d_s_match_yn            46404 non-null  bool    
 3   h_l_match_yn            46404 non-null  bool    
 4   h_m_match_yn            46404 non-null  bool    
 5   h_s_match_yn            46404 non-null  bool    
 6   person_attribute_a      46404 non-null  category
 7   person_attribute_a_1    46404 non-null  category
 8   person_attribute_b      46404 non-null  category
 9   person_prefer_c         46404 non-null  category
 10  person_prefer_d_1       46404 non-null  object  
 11  person_prefer_d_2       46404 non-null  object  
 12  person_prefer_d_3       46404 non-null  object  
 13  person_prefer_e         46404 non-null  category
 14  person_prefer_h_1     

In [ ]:
cat_cols = train.columns[train.dtypes == 'category'].to_list()

In [ ]:
cat_cols[:5]

In [ ]:
cat_cols[5:]

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
sns.lmplot( x='person_attribute_a' , y='person_attribute_a_1', data=train, fit_reg=False, scatter_kws={"s":30}) 
plt.title('k-mean')
plt.xlabel('x')
plt.ylabel('y')

In [ ]:
train

In [ ]:
cat_person_train = train[cat_cols[:5]]

In [20]:
one_cols = ["person_attribute_a","person_prefer_c"]
for i in one_cols:
    onehot = pd.get_dummies(cat_person_train[i])
    onehot = onehot.astype('bool')
    onehot.rename(columns = lambda x:"one_hot_"+str(i)+"_"+str(x), inplace = True)
    cat_person_train = pd.concat([cat_person_train,onehot],axis=1)

In [21]:
cat_person_train

,person_attribute_a,person_attribute_a_1,person_attribute_b,person_prefer_c,person_prefer_e,one_hot_person_attribute_a_1,one_hot_person_attribute_a_2,one_hot_person_prefer_c_1,one_hot_person_prefer_c_2,one_hot_person_prefer_c_3,one_hot_person_prefer_c_4,one_hot_person_prefer_c_5
0,1,4,3,5,8,True,False,False,False,False,False,True
1,1,3,4,1,4,True,False,True,False,False,False,False
2,2,0,3,5,3,False,True,False,False,False,False,True
3,2,0,2,5,3,False,True,False,False,False,False,True
4,1,3,4,5,4,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
501946,1,1,2,2,3,True,False,False,True,False,False,False
501947,1,6,2,1,4,True,False,True,False,False,False,False
501948,1,7,4,1,4,True,False,True,False,False,False,False
501949,1,1,2,1,4,True,False,True,False,False,False,False


In [22]:
cat_content_train = train[cat_cols[5:]]

In [23]:
one_cols = ["contents_attribute_i","contents_attribute_a","contents_attribute_j_1","contents_attribute_j","contents_attribute_c","contents_attribute_k","contents_attribute_m"]
for i in one_cols:
    onehot = pd.get_dummies(cat_content_train[i])
    onehot = onehot.astype('bool')
    onehot.rename(columns = lambda x:"one_hot_"+str(i)+"_"+str(x), inplace = True)
    cat_content_train = pd.concat([cat_content_train,onehot],axis=1)

In [24]:
cat_content_train

,contents_attribute_i,contents_attribute_a,contents_attribute_j_1,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_m,contents_attribute_e,one_hot_contents_attribute_i_1,one_hot_contents_attribute_i_2,...,one_hot_contents_attribute_c_2,one_hot_contents_attribute_c_3,one_hot_contents_attribute_c_4,one_hot_contents_attribute_k_1,one_hot_contents_attribute_k_2,one_hot_contents_attribute_m_1,one_hot_contents_attribute_m_2,one_hot_contents_attribute_m_3,one_hot_contents_attribute_m_4,one_hot_contents_attribute_m_5
0,3,3,10,2,1,2,1,4,False,False,...,False,False,False,False,True,True,False,False,False,False
1,1,3,5,1,1,2,1,4,True,False,...,False,False,False,False,True,True,False,False,False,False
2,3,1,10,2,1,1,1,4,False,False,...,False,False,False,True,False,True,False,False,False,False
3,1,3,5,1,1,2,5,3,True,False,...,False,False,False,False,True,False,False,False,False,True
4,1,1,10,2,1,2,1,4,True,False,...,False,False,False,False,True,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501946,3,3,5,1,1,2,1,5,False,False,...,False,False,False,False,True,True,False,False,False,False
501947,3,3,10,2,1,2,1,4,False,False,...,False,False,False,False,True,True,False,False,False,False
501948,3,1,5,1,1,2,2,7,False,False,...,False,False,False,False,True,False,True,False,False,False
501949,3,2,5,1,1,2,3,4,False,False,...,False,False,False,False,True,False,False,True,False,False


In [25]:
n_cluster = 20

In [26]:
person_values = cat_person_train.values

In [ ]:
person_cluster_df = pd.DataFrame()
kmeans=KMeans(n_clusters=n_cluster).fit(person_values)
person_cluster_df['cluster'] = kmeans.labels_ 
for i in range(20):
    kmeans=KMeans(n_clusters=n_cluster).fit(person_values)
    person_cluster_df['cluster'+str(i)] = kmeans.labels_
    person_cluster_df['cluster_temp'] = kmeans.labels_
    cluster_unique = list(range(n_cluster))
    cluster_unique2 = list(range(n_cluster))
    for j in cluster_unique2:
        if_df = person_cluster_df[person_cluster_df['cluster'+str(i)]==j]
        if if_df["cluster"].mode()[0] in cluster_unique:
            if_df["cluster_temp"] = if_df["cluster"].mode()[0]
            cluster_unique.remove(if_df["cluster"].mode()[0])
    person_cluster_df['cluster'+str(i)] = person_cluster_df['cluster_temp']
person_cluster_df = person_cluster_df.drop("cluster_temp",axis=1)
cat_person_train["person_cluster"] = person_cluster_df.mode(axis=1)[0]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\swm\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\swm\AppData\Local\Temp/ipykernel_7992/457492599.py", line 5, in <module>
    kmeans=KMeans(n_clusters=n_cluster).fit(person_values)
  File "c:\users\swm\appdata\local\programs\python\python39\lib\site-packages\sklearn\cluster\_kmeans.py", line 1023, in fit
    labels, inertia, centers, n_iter_ = kmeans_single(
  File "c:\users\swm\appdata\local\programs\python\python39\lib\site-packages\sklearn\cluster\_kmeans.py", line 392, in _kmeans_single_elkan
    center_half_distances = euclidean_distances(centers_new) / 2
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\swm\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py", line

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\swm\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\swm\AppData\Local\Temp/ipykernel_7992/457492599.py", line 5, in <module>
    kmeans=KMeans(n_clusters=n_cluster).fit(person_values)
  File "c:\users\swm\appdata\local\programs\python\python39\lib\site-packages\sklearn\cluster\_kmeans.py", line 1023, in fit
    labels, inertia, centers, n_iter_ = kmeans_single(
  File "c:\users\swm\appdata\local\programs\python\python39\lib\site-packages\sklearn\cluster\_kmeans.py", line 392, in _kmeans_single_elkan
    center_half_distances = euclidean_distances(centers_new) / 2
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\swm\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py", line

In [27]:
person_cluster_df

,cluster,cluster0,cluster_temp
0,11,17,17
1,2,4,4
2,8,9,9
3,8,12,12
4,4,15,15
...,...,...,...
501946,14,13,13
501947,7,0,0
501948,7,0,0
501949,14,13,13


ERROR! Session/line number was not unique in database. History logging moved to new session 250


In [28]:
cat_person_train

,person_attribute_a,person_attribute_a_1,person_attribute_b,person_prefer_c,person_prefer_e,one_hot_person_attribute_a_1,one_hot_person_attribute_a_2,one_hot_person_prefer_c_1,one_hot_person_prefer_c_2,one_hot_person_prefer_c_3,one_hot_person_prefer_c_4,one_hot_person_prefer_c_5
0,1,4,3,5,8,True,False,False,False,False,False,True
1,1,3,4,1,4,True,False,True,False,False,False,False
2,2,0,3,5,3,False,True,False,False,False,False,True
3,2,0,2,5,3,False,True,False,False,False,False,True
4,1,3,4,5,4,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
501946,1,1,2,2,3,True,False,False,True,False,False,False
501947,1,6,2,1,4,True,False,True,False,False,False,False
501948,1,7,4,1,4,True,False,True,False,False,False,False
501949,1,1,2,1,4,True,False,True,False,False,False,False


In [29]:
content_values = cat_content_train.values 

In [30]:
content_cluster_df = pd.DataFrame()
kmeans=KMeans(n_clusters=n_cluster).fit(content_values)
content_cluster_df['cluster'] = kmeans.labels_
for i in range(20):
    kmeans=KMeans(n_clusters=n_cluster).fit(content_values)
    content_cluster_df['cluster'+str(i)] = kmeans.labels_
    content_cluster_df['cluster_temp'] = kmeans.labels_
    cluster_unique = list(range(n_cluster))
    cluster_unique2 = list(range(n_cluster))
    for j in cluster_unique2:
        if_df = content_cluster_df[content_cluster_df['cluster'+str(i)]==j]
        if if_df["cluster"].mode()[0] in cluster_unique:
            if_df["cluster_temp"] = if_df["cluster"].mode()[0]
            cluster_unique.remove(if_df["cluster"].mode()[0])
    content_cluster_df['cluster'+str(i)] = content_cluster_df['cluster_temp']
content_cluster_df = content_cluster_df.drop("cluster_temp",axis=1)
cat_content_train["content_cluster"] = content_cluster_df.mode(axis=1)[0]

KeyboardInterrupt: 

In [ ]:
content_cluster_df

In [ ]:
cat_content_train

In [ ]:
fold = StratifiedKFold(n_splits=6, shuffle = True, random_state=62)

In [ ]:
y = pd.read_csv('train.csv')
y = y["target"]

In [ ]:
train_2 = train[cat_cols]

In [ ]:
one_cols = ["person_attribute_a","person_prefer_c","contents_attribute_i","contents_attribute_a","contents_attribute_j_1","contents_attribute_j","contents_attribute_c","contents_attribute_k","contents_attribute_m"]
for i in one_cols:
    onehot = pd.get_dummies(train_2[i])
    onehot = onehot.astype('bool')
    onehot.rename(columns = lambda x:"one_hot_"+str(i)+"_"+str(x), inplace = True)
    train_2 = pd.concat([train_2,onehot],axis=1)

In [ ]:
catc_model = CatBoostClassifier(cat_features=cat_cols) 
catc_score = cross_val_score(catc_model , train_2 , y ,cv=fold,scoring=make_scorer(class_score))
catc_score = np.mean(catc_score)

In [ ]:
catc_score

In [ ]:
train_3 = pd.concat([cat_person_train,cat_content_train],axis=1)

In [ ]:
catc_model2 = CatBoostClassifier(cat_features=cat_cols)
catc_model2.fit(train_3,y)
catc_score2 = cross_val_score(catc_model2 , train_3 , y ,cv=fold,scoring=make_scorer(class_score))
print(catc_score2)
catc_score2 = np.mean(catc_score2)

In [ ]:
catc_score2

In [ ]:
cluster_csv = pd.concat([train_3["person_cluster"],train_3["content_cluster"]],axis=1)

In [ ]:
cluster_csv

In [ ]:
cluster_csv.to_csv("cluster.csv",index=False)

In [41]:
cat_cols = test.columns[test.dtypes == 'category'].to_list()
cat_person_test = test[cat_cols[:5]]
one_cols = ["person_attribute_a","person_prefer_c"]
for i in one_cols:
    onehot = pd.get_dummies(cat_person_test[i])
    onehot = onehot.astype('bool')
    onehot.rename(columns = lambda x:"one_hot_"+str(i)+"_"+str(x), inplace = True)
    cat_person_test = pd.concat([cat_person_test,onehot],axis=1)
cat_content_test = test[cat_cols[5:]]
one_cols = ["contents_attribute_i","contents_attribute_a","contents_attribute_j_1","contents_attribute_j","contents_attribute_c","contents_attribute_k","contents_attribute_m"]
for i in one_cols:
    onehot = pd.get_dummies(cat_content_test[i])
    onehot = onehot.astype('bool')
    onehot.rename(columns = lambda x:"one_hot_"+str(i)+"_"+str(x), inplace = True)
    cat_content_test = pd.concat([cat_content_test,onehot],axis=1)
n_cluster = 20
person_values = cat_person_test.values
person_cluster_df = pd.DataFrame()
kmeans=KMeans(n_clusters=n_cluster).fit(person_values)
person_cluster_df['cluster'] = kmeans.labels_ 
for i in range(20):
    kmeans=KMeans(n_clusters=n_cluster).fit(person_values)
    person_cluster_df['cluster'+str(i)] = kmeans.labels_
    person_cluster_df['cluster_temp'] = kmeans.labels_
    cluster_unique = list(range(n_cluster))
    cluster_unique2 = list(range(n_cluster))
    for j in cluster_unique2:
        if_df = person_cluster_df[person_cluster_df['cluster'+str(i)]==j]
        if if_df["cluster"].mode()[0] in cluster_unique:
            if_df["cluster_temp"] = if_df["cluster"].mode()[0]
            cluster_unique.remove(if_df["cluster"].mode()[0])
    person_cluster_df['cluster'+str(i)] = person_cluster_df['cluster_temp']
person_cluster_df = person_cluster_df.drop("cluster_temp",axis=1)
cat_person_test["person_cluster"] = person_cluster_df.mode(axis=1)[0]
content_values = cat_content_test.values 
content_cluster_df = pd.DataFrame()
kmeans=KMeans(n_clusters=n_cluster).fit(content_values)
content_cluster_df['cluster'] = kmeans.labels_
for i in range(20):
    kmeans=KMeans(n_clusters=n_cluster).fit(content_values)
    content_cluster_df['cluster'+str(i)] = kmeans.labels_
    content_cluster_df['cluster_temp'] = kmeans.labels_
    cluster_unique = list(range(n_cluster))
    cluster_unique2 = list(range(n_cluster))
    for j in cluster_unique2:
        if_df = content_cluster_df[content_cluster_df['cluster'+str(i)]==j]
        if if_df["cluster"].mode()[0] in cluster_unique:
            if_df["cluster_temp"] = if_df["cluster"].mode()[0]
            cluster_unique.remove(if_df["cluster"].mode()[0])
    content_cluster_df['cluster'+str(i)] = content_cluster_df['cluster_temp']
content_cluster_df = content_cluster_df.drop("cluster_temp",axis=1)
cat_content_test["content_cluster"] = content_cluster_df.mode(axis=1)[0]

cat_content_test

,contents_attribute_i,contents_attribute_a,contents_attribute_j_1,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_m,contents_attribute_e,one_hot_contents_attribute_i_1,one_hot_contents_attribute_i_2,...,one_hot_contents_attribute_c_3,one_hot_contents_attribute_c_4,one_hot_contents_attribute_k_1,one_hot_contents_attribute_k_2,one_hot_contents_attribute_m_1,one_hot_contents_attribute_m_2,one_hot_contents_attribute_m_3,one_hot_contents_attribute_m_4,one_hot_contents_attribute_m_5,content_cluster
0,1,3,10,2,1,2,1,5,True,False,...,False,False,False,True,True,False,False,False,False,17.0
1,1,3,5,1,1,2,1,4,True,False,...,False,False,False,True,True,False,False,False,False,4.0
2,3,3,3,1,1,2,3,4,False,False,...,False,False,False,True,False,False,True,False,False,18.0
3,3,3,5,1,1,2,5,3,False,False,...,False,False,False,True,False,False,False,False,True,5.0
4,3,1,5,1,1,1,1,4,False,False,...,False,False,True,False,True,False,False,False,False,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46399,3,3,5,1,1,2,1,5,False,False,...,False,False,False,True,True,False,False,False,False,2.0
46400,3,3,10,2,1,2,1,4,False,False,...,False,False,False,True,True,False,False,False,False,3.0
46401,3,3,10,2,1,2,1,4,False,False,...,False,False,False,True,True,False,False,False,False,3.0
46402,3,1,5,1,3,2,1,5,False,False,...,True,False,False,True,True,False,False,False,False,4.0


In [42]:
submission = pd.read_csv('sample_submission.csv')
submission = submission.drop("target",axis=1)
submission["person_cluster"] = cat_person_test["person_cluster"]
submission["content_cluster"] = cat_content_test["content_cluster"]
submission.to_csv("test_cluster.csv",index=False)